### Импортим необходимые инструменты

In [2]:
import json
import nltk
#nltk.download('punkt')
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

### Считываем данные

In [3]:
n = 10000 # кол-во обрабатываемых отзывов
all_texts = [] # выделяем текст отзывов
all_overalls = np.zeros((n, 1)) #выделяем оценки
# считываем данные из файла
j = 0
with open('Beauty_5.json', 'r') as f:
    for line in f:
        if j == n:
            break
        s = json.loads(line)
        all_texts.append(s["reviewText"])
        all_overalls[j] = s["overall"]
        j = j + 1

In [4]:
# бинаризуем оценки
binarized_overalls = (all_overalls > 3).astype(int) 

### Векторизация необработанных текстов

In [5]:
# векторизация необработанных текстов
# TF-IDF

tfidf_vectorizer = TfidfVectorizer()
untreated_words = tfidf_vectorizer.fit_transform(all_texts)

# Show as a pandas DataFrame
feature_names = tfidf_vectorizer.get_feature_names()
df1 = pd.DataFrame(untreated_words.toarray(), columns = feature_names)

In [6]:
df1

,00,000,01,03,05,06,0639v,07,08,09,...,zits,zo,zoloft,zone,zones,zoya,zoyas,zumba,zuz,zuzu
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Предобработка текстов

Цели предобработки: сделать набор слов удобным для изучения, таким, чтобы машине было проще понять, как сопоставить слова и оценки. Для этого нужно выбрать не только самые важные по смыслу слова, но и представить их в такой форме, чтобы сократить расход памяти и время анализа.\
Что мы для этого можем сделать?\
Во-первых, разбить все на слова, а дальше работать с ними.\
1) привести все буквы к нижнему регистру (понятно, что регистр нам не важен)\
2) удалить знаки пунктуации (они не дают никакой информации, для нас это просто мусор)\
3) удалить числа и слова, содержащие цифры (могут относиться к чему угодно не по теме, думаю ими можно пренебречь)\
4) удалить слова с нижним подчеркиванием (их сложно свести к обычным и они встречаются редко)\
5) удалить стоп-слова\
6) удалить слова, которые встречаются слишком редко или слишком часто (первые будут только увеличивать размер таблиц, не принося особой пользы, а вторые обычно не несут смысловой нагрузки)\
7) провести стемминг (различные формы слова не так значительны как его основа). \ Сначала я хотела проводить лемматизацию, но она оставляет такие слова как "worked", "working", "works", и я посчитала, что такие формы можно игнорировать.
Пункты 1,5-7 можно объединить с tf-idf векторайзером.\
*Еще я заметила там слова с большим количеством повторяющихся букв, но не знаю что с ними сделать. По идее они тоже мешаются.

In [7]:
words = [] # массив массивов слов из каждого предложения после предобработки
for i in range(len(all_texts)):
    wordLists = re.sub(r"_", "", all_texts[i]) # удаление нижнего подчёркивания
    wordList = re.sub("[^\w]", " ",  wordLists).split() # удаление пунктуации
    # удаление слов, содержащих цифры
    for j in range(len(wordList)):
        for k in range(len(wordList[j])):
            if wordList[j][k].isdigit():
                wordList[j] = ''
                break
    for j in range(len(wordList)):
        #wordList[j] = WordNetLemmatizer().lemmatize(wordList[j], wordnet.VERB) # лемматизация
        wordList[j] = PorterStemmer().stem(wordList[j]) # стемминг
    words.append(wordList)

### Векторизация обработанных текстов

In [8]:
# создадим массив предложений обработанных слов
sentences = []
for i in range(len(words)):
    s = ''
    for j in range(len(words[i])):
        s = s + words[i][j] + ' '
    sentences.append(s)

# TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words = 'english', min_df = 0.01, max_df = 0.99)
treated_words = tfidf_vectorizer.fit_transform(sentences)

# Show as a pandas DataFrame
feature_names = tfidf_vectorizer.get_feature_names()
df2 = pd.DataFrame(treated_words.toarray(), columns = feature_names)
#df2["class"] = binarized_overalls

In [9]:
df2

,abl,absolut,absorb,acid,acn,actual,ad,add,addit,afford,...,wonder,work,worri,wors,worth,wouldn,wrinkl,wrong,ye,year
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.200441,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Разбиение на выборки

Разобьем полученные выборки на обучающие и тестовые в соотношении 7:3.
Затем обе обучающие (для обработанных и для необработанных текстов) разобьем на 5 примерно равных частей (фолдов). Далее будем проводить 5-fold кросс-валидацию.

In [83]:
# необработанные
untreated_words_train = untreated_words[:7000]
untreated_words_test = untreated_words[7000:]

# обработанные
treated_words_train = treated_words[:7000]
treated_words_test = treated_words[7000:]

# оценки
overalls_train = binarized_overalls[:7000]
overalls_test = binarized_overalls[7000:]

from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

for train_index, test_index in kf.split(untreated_words_train):
    print("TRAIN_1:", train_index, "TEST_1:", test_index)
    
for train_index, test_index in kf.split(treated_words_train):
    print("TRAIN_2:", train_index, "TEST_2:", test_index)

TRAIN_1: [1400 1401 1402 ... 6997 6998 6999] TEST_1: [   0    1    2 ... 1397 1398 1399]
TRAIN_1: [   0    1    2 ... 6997 6998 6999] TEST_1: [1400 1401 1402 ... 2797 2798 2799]
TRAIN_1: [   0    1    2 ... 6997 6998 6999] TEST_1: [2800 2801 2802 ... 4197 4198 4199]
TRAIN_1: [   0    1    2 ... 6997 6998 6999] TEST_1: [4200 4201 4202 ... 5597 5598 5599]
TRAIN_1: [   0    1    2 ... 5597 5598 5599] TEST_1: [5600 5601 5602 ... 6997 6998 6999]
TRAIN_2: [1400 1401 1402 ... 6997 6998 6999] TEST_2: [   0    1    2 ... 1397 1398 1399]
TRAIN_2: [   0    1    2 ... 6997 6998 6999] TEST_2: [1400 1401 1402 ... 2797 2798 2799]
TRAIN_2: [   0    1    2 ... 6997 6998 6999] TEST_2: [2800 2801 2802 ... 4197 4198 4199]
TRAIN_2: [   0    1    2 ... 6997 6998 6999] TEST_2: [4200 4201 4202 ... 5597 5598 5599]
TRAIN_2: [   0    1    2 ... 5597 5598 5599] TEST_2: [5600 5601 5602 ... 6997 6998 6999]


### Описание моделей

Создадим набор параметров моделей, которые мы будем тестировать. Нужные параметры: \
1. Penalty: none/l2 - регуляризация
2. C: 0.1, 0.5, 1, 1.5, 2, 5 - коэффициент регуляризации
3. Class_weight: none/balanced - веса классов
4. Solver: {‘newton-cg’, ‘lbfgs’, ‘liblinear’, ‘sag’, ‘saga’} - оптимизатор

А также создадим массив для оценок с кросс-валидации - scores (140 * 7) - первые 5 столбцов - сами оценки, 6-й - мат. ожидание, 7-й - дисперсия.

In [93]:
p = [
    ['none', 'l2'],
    [0.1, 0.5, 1, 1.5, 2, 5],
    ['', 'balanced'],
    ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
]

params = []
import itertools
for element in itertools.product(*p):
    params.append(element)

for i in range(10, 60):
    params.pop(i)
    
for i in range(35):
    if params[i][3] == 'liblinear':
        params.pop(i)
    
model_params = pd.DataFrame(params, columns = ('Penalty', 'C', 'Class_weight', 'Solver'))
model_params

,Penalty,C,Class_weight,Solver
0,none,0.1,,newton-cg
1,none,0.1,,lbfgs
2,none,0.1,,sag
3,none,0.1,,saga
4,none,0.1,balanced,newton-cg
...,...,...,...,...
57,l2,5.0,balanced,newton-cg
58,l2,5.0,balanced,lbfgs
59,l2,5.0,balanced,liblinear
60,l2,5.0,balanced,sag


### Обучение моделей на необработанных текстах

In [107]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
s = np.zeros((62, 7))

for i in range(62):
    model = LogisticRegression(penalty = params[i][0],
                               C = params[i][1],
                               class_weight = params[i][2],
                               solver = params[i][3])
    
    k = 0
    for train_index, test_index in kf.split(untreated_words_train):
        x_train, x_test = untreated_words_train[train_index], untreated_words_train[test_index]
        y_train, y_test = overalls_train[train_index], overalls_train[test_index]
        
        model.fit(x_train, y_train)
        prediction = model.predict(x_test)
        s[i][k] = accuracy_score(y_test, prediction)
        k += 1
    s[i][5] = np.mean(s[i][:5]) # считаем среднее
    s[i][6] = np.var(s[i][:5]) # считаем дисперсию
        
scores = pd.DataFrame(s, columns = ('s1', 's2', 's3', 's4', 's5', 'Es', 'Ds'))
scores
    

,s1,s2,s3,s4,s5,Es,Ds
0,0.827857,0.795714,0.832143,0.818571,0.810714,0.817000,0.000168
1,0.833571,0.800000,0.840000,0.816429,0.825000,0.823000,0.000195
2,0.837857,0.815714,0.835714,0.830000,0.823571,0.828571,0.000066
3,0.842143,0.813571,0.844286,0.834286,0.828571,0.832571,0.000122
4,0.827857,0.792857,0.827857,0.813571,0.810000,0.814429,0.000169
...,...,...,...,...,...,...,...
57,0.840714,0.832857,0.849286,0.844286,0.817857,0.837000,0.000120
58,0.840714,0.832857,0.849286,0.844286,0.817857,0.837000,0.000120
59,0.840714,0.832857,0.849286,0.844286,0.817857,0.837000,0.000120
60,0.840714,0.832857,0.849286,0.844286,0.817857,0.837000,0.000120


In [108]:
scores.sort_values(by = 'Es')

,s1,s2,s3,s4,s5,Es,Ds
29,0.770000,0.716429,0.789286,0.800714,0.786429,0.772571,0.000884
28,0.770000,0.716429,0.789286,0.800714,0.786429,0.772571,0.000884
30,0.789286,0.782857,0.806429,0.786429,0.810000,0.795000,0.000122
31,0.788571,0.784286,0.807143,0.786429,0.810000,0.795286,0.000120
33,0.813571,0.755714,0.830714,0.835714,0.830714,0.813286,0.000885
...,...,...,...,...,...,...,...
53,0.855714,0.821429,0.862143,0.865714,0.852857,0.851571,0.000248
52,0.856429,0.821429,0.862143,0.865000,0.852857,0.851571,0.000245
54,0.856429,0.822143,0.861429,0.865000,0.852857,0.851571,0.000234
55,0.856429,0.821429,0.862143,0.865000,0.852857,0.851571,0.000245


###  Выбор модели для необработанных текстов
Самый большой показатель в среднем у модели 56, и из топ-5 отсортированных по среднему у нее наименьшая дисперсия - выбираем ее.

In [109]:
model = LogisticRegression(penalty = params[56][0],
                               C = params[56][1],
                               class_weight = params[56][2],
                               solver = params[56][3])
model.fit(untreated_words_train, overalls_train)
prediction = model.predict(untreated_words_test)
score = accuracy_score(overalls_test, prediction)
print(score)

0.845


### Обучение моделей на обработанных текстах

In [110]:
s2 = np.zeros((62, 7))

for i in range(62):
    model = LogisticRegression(penalty = params[i][0],
                               C = params[i][1],
                               class_weight = params[i][2],
                               solver = params[i][3])
    
    k = 0
    for train_index, test_index in kf.split(treated_words_train):
        x_train, x_test = treated_words_train[train_index], treated_words_train[test_index]
        y_train, y_test = overalls_train[train_index], overalls_train[test_index]
        
        model.fit(x_train, y_train)
        prediction = model.predict(x_test)
        s[i][k] = accuracy_score(y_test, prediction)
        k += 1
    s[i][5] = np.mean(s[i][:5]) # считаем среднее
    s[i][6] = np.var(s[i][:5]) # считаем дисперсию
        
scores2 = pd.DataFrame(s, columns = ('s1', 's2', 's3', 's4', 's5', 'Es', 'Ds'))
scores2

,s1,s2,s3,s4,s5,Es,Ds
0,0.825000,0.804286,0.838571,0.823571,0.829286,0.824143,0.000126
1,0.825000,0.805000,0.838571,0.823571,0.828571,0.824143,0.000119
2,0.825000,0.804286,0.838571,0.824286,0.829286,0.824286,0.000126
3,0.825000,0.804286,0.837857,0.823571,0.829286,0.824000,0.000122
4,0.789286,0.773571,0.778571,0.767143,0.764286,0.774571,0.000079
...,...,...,...,...,...,...,...
57,0.795000,0.785714,0.790000,0.782143,0.782857,0.787143,0.000023
58,0.795000,0.785714,0.790000,0.782143,0.782857,0.787143,0.000023
59,0.795000,0.785714,0.790000,0.782143,0.782857,0.787143,0.000023
60,0.795000,0.785714,0.790000,0.782143,0.782857,0.787143,0.000023


In [111]:
scores2.sort_values(by = 'Es')

,s1,s2,s3,s4,s5,Es,Ds
30,0.787143,0.765714,0.779286,0.755714,0.780000,0.773571,0.000128
31,0.787143,0.765714,0.779286,0.755714,0.780000,0.773571,0.000128
22,0.789286,0.773571,0.778571,0.767143,0.764286,0.774571,0.000079
18,0.789286,0.773571,0.778571,0.767143,0.764286,0.774571,0.000079
14,0.789286,0.773571,0.778571,0.767143,0.764286,0.774571,0.000079
...,...,...,...,...,...,...,...
54,0.837143,0.812143,0.850714,0.846429,0.836429,0.836571,0.000179
56,0.836429,0.812143,0.851429,0.846429,0.836429,0.836571,0.000183
53,0.837143,0.812143,0.850714,0.846429,0.837143,0.836714,0.000179
55,0.837143,0.812143,0.850714,0.847143,0.836429,0.836714,0.000182


###  Выбор модели для обработанных текстов
Самый большой показатель в среднем у моделей 52, 53, 55, и из топ-5 отсортированных по среднему у 53 наименьшая дисперсия - выбираем ее.

In [112]:
model = LogisticRegression(penalty = params[53][0],
                               C = params[53][1],
                               class_weight = params[53][2],
                               solver = params[53][3])
model.fit(treated_words_train, overalls_train)
prediction = model.predict(treated_words_test)
score = accuracy_score(overalls_test, prediction)
print(score)

0.8236666666666667
